In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 29kB/s 
     |████████████████████████████████| 512kB 39.9MB/s 
     |████████████████████████████████| 3.8MB 17.9MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=7eeeefbd9eded6765a878de82f7775c0648068e293fc2e5a658bb2003451d518
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [ ]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [ ]:
from keras.models import load_model
m = load_model("/content/drive/MyDrive/model/model-use-vncore-4500-1-1-1.h5")






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [ ]:
import numpy as np
from numpy import argmax

In [ ]:
char_encode = np.loadtxt('/content/drive/MyDrive/fast_text/val/char_encode.txt')
print(char_encode.shape)
char_encode = char_encode.reshape(int(char_encode.shape[0]/42), 42, char_encode.shape[1])
print(char_encode.shape)

(18900, 32)
(450, 42, 32)


In [ ]:
# chuyển từ char encode sang char embedding
LEN_OF_VOCAB = 137
shape = char_encode.shape
char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
for i in range(shape[0]):
  for j in range(shape[1]):
    for k in range(shape[2]):
      char_int = char_encode[i,j,k]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,k,:] = onehot

print(char_embedd.shape)

(450, 42, 32, 137)


In [ ]:
tag = np.loadtxt('/content/drive/MyDrive/fast_text/val/tag_embedd.txt')
tag = tag.reshape(int(tag.shape[0]/42), 42, tag.shape[1])
print(tag.shape)

(450, 42, 13)


In [ ]:
word = np.loadtxt('/content/drive/MyDrive/fast_text/val/word_embedd.txt')
word = word.reshape(int(word.shape[0]/42), 42, word.shape[1])
print(word.shape)

(450, 42, 300)


In [ ]:
y_pred = m.predict([char_embedd, word])
y_true = tag
print(y_pred.shape)
print(y_true.shape)

(450, 42, 13)
(450, 42, 13)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
print(y_pred.shape)
y_pred = np.argmax(y_pred,axis=1)
print(y_pred.shape)
y_pred = y_pred.reshape(int(y_pred.shape[0]/42), 42)
print(y_pred.shape)

(18900, 13)
(18900,)
(450, 42)


In [ ]:
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1], y_true.shape[2])
print(y_true.shape)
y_true = np.argmax(y_true,axis=1)
print(y_true.shape)
y_true = y_true.reshape(int(y_true.shape[0]/42), 42)
print(y_true.shape)

(18900, 13)
(18900,)
(450, 42)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def arr_tag_in_a_tag_sen(sen_pred, sen_true, tag):
  j=0
  bat_dau=-1
  ket_thuc=-1
  while j < len(sen_pred):
    if sen_true[j]==tag:
      bat_dau=j
      ket_thuc=j
      while sen_true[ket_thuc+1]==1:
        ket_thuc=ket_thuc+1
      ket_thuc=ket_thuc+1
      break
    else:
        j=j+1

  if bat_dau==-1 and ket_thuc==-1:
    return 0
  
  a = sen_pred[bat_dau:ket_thuc]
  b = sen_true[bat_dau:ket_thuc]
  a = np.array(a)
  b = np.array(b)
  if (a==b).all():
    return 1
  else:
    return 0

In [ ]:
from sklearn.metrics import accuracy_score
acc=0
false=0

homenumber_true = 0
street_true = 0 
ward_true = 0
district_true = 0
province_true = 0
country_true = 0
postcode_true = 0
ner_true = 0
obj_true = 0
obj_feature_true = 0
pre_true = 0

homenumber_false = 0
street_false = 0
ward_false = 0
district_false = 0
province_false = 0
country_false = 0
postcode_false = 0
ner_false = 0
obj_false = 0
obj_feature_false = 0
pre_false = 0

In [ ]:
for i in range(len(y_true)):
  sen_pred = y_pred[i]
  sen_true = y_true[i]
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 1)==1:
    homenumber_true+=1
  else:
    homenumber_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 1)==1:
    homenumber_true+=1
  else:
    homenumber_false+=1

  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 2)==1:
    street_true+=1
  else:
    street_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 3)==1:
    ward_true+=1
  else:
    ward_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 4)==1:
    district_true+=1
  else:
    district_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 5)==1:
    province_true+=1
  else:
    province_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 6)==1:
    country_true+=1
  else:
    country_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 7)==1:
    postcode_true+=1
  else:
    postcode_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 8)==1:
    ner_true+=1
  else:
    ner_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 9)==1:
    obj_true+=1
  else:
    obj_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 10)==1:
    obj_feature_true+=1
  else:
    obj_feature_false+=1

  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 11)==1:
    pre_true+=1
  else:
    pre_false+=1

In [ ]:
acc = homenumber_true+ street_true+ ward_true+ district_true+ province_true+ country_true+ postcode_true+ ner_true+ obj_true+ obj_feature_true+ pre_true
false = homenumber_false+ street_false+ ward_false+ district_false+ province_false+ country_false+ postcode_false+ ner_false+ obj_false+ obj_feature_false+ pre_false
print(acc)
print(false)
print('accuracy = {}'.format(acc/(acc+false)) )
ti_le_dung = format(acc/(acc+false)*100, '.2f')
print(ti_le_dung)

print("homenumber : {} true, {} false".format(homenumber_true, homenumber_false))
print("street : {} true, {} false".format(street_true, street_false))
print("ward : {} true, {} false".format(ward_true, ward_false))
print("district : {} true, {} false".format(district_true, district_false))
print("province : {} true, {} false".format(province_true, province_false))
print("country : {} true, {} false".format(country_true, country_false))
print("postcode : {} true, {} false".format(postcode_true, postcode_false))
print("ner : {} true, {} false".format(ner_true, ner_false))
print("obj : {} true, {} false".format(obj_true, obj_false))
print("obj_feature : {} true, {} false".format(obj_feature_true, obj_feature_false))
print("pre : {} true, {} false".format(pre_true, pre_false))


2717
2683
accuracy = 0.5031481481481481
50.31
homenumber : 490 true, 410 false
street : 295 true, 155 false
ward : 145 true, 305 false
district : 300 true, 150 false
province : 300 true, 150 false
country : 170 true, 280 false
postcode : 300 true, 150 false
ner : 143 true, 307 false
obj : 292 true, 158 false
obj_feature : 0 true, 450 false
pre : 282 true, 168 false


In [ ]:
dict = []
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1])
y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1])
for element in y_true:
  if element not in dict:
    dict.append(element)
# print(dict)
dict.sort()
print(dict)

cnf_matrix = confusion_matrix(y_true, y_pred, labels=dict)
for i in range(len(cnf_matrix)):
  for j in range(len(cnf_matrix)):
    cnf_matrix[i][j] = round(cnf_matrix[i][j], 3)
print(cnf_matrix)

arr_to_cal_predict = []
for i in range(len(cnf_matrix)):
  tong= 0
  for j in range(len(cnf_matrix)):
    tong+=cnf_matrix[j][i]
  arr_to_cal_predict.append(tong)
print(arr_to_cal_predict)
# lấy độ predict
predict = []
for i in range(len(cnf_matrix)):
  tong=cnf_matrix[i][i]/arr_to_cal_predict[i]
  predict.append(tong)

print("Độ đo P của từng tag: ")
print(predict)
print(len(predict))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[[13497     0     0     0     0     0     0     0     0     0     0     0
      0]
 [    0   301    10     0     0     0     0     0     0     0     0     0
      0]
 [    0     1  1127    26     1     0     0     0     0     0     0     0
      0]
 [    0     0     0   145     0     0     0     0     0     0     0     1
      0]
 [    0     0     0     0   300     0     0     0     0     0     0     0
      0]
 [    0     0     0     0     0   300     0     0     0     0     0     0
      0]
 [    0     0     0     0     0     0   170     0     0     0     0     0
      0]
 [    0     0     0     0     0     0     0   300     0     0     0     0
      0]
 [    1     0     3     0     1     0     0     3   474     3     0     5
      6]
 [    0     3     0     0     0     0     0     0     3   556     0     4
      6]
 [    0    19    13     0     0     0     0     1    43   252     0     3
      4]
 [    0     1    13     0     0     0     0 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in long_scalars


In [ ]:
normalized_confusion_matrix = cnf_matrix/cnf_matrix.sum(axis = 1, keepdims = True)
for i in range(len(normalized_confusion_matrix)):
  for j in range(len(normalized_confusion_matrix)):
    normalized_confusion_matrix[i][j] = round(normalized_confusion_matrix[i][j], 3)
print(normalized_confusion_matrix)

# lấy độ recall
recall = []
for i in range(len(normalized_confusion_matrix)):
  recall.append(normalized_confusion_matrix[i][i])

print("Độ đo R của từng tag: ")
print(recall)
print(len(recall))

[[1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.968 0.032 0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.001 0.976 0.023 0.001 0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.993 0.    0.    0.    0.    0.    0.    0.    0.007
  0.   ]
 [0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.
  0.   ]
 [0.002 0.    0.006 0.    0.002 0.    0.    0.006 0.956 0.006 0.    0.01
  0.012]
 [0.    0.005 0.    0.    0.    0.    0.    0.    0.005 0.972 0.    0.007
  0.01 ]
 [0.    0.057 0.039 0.    0.    0.    0.    0.003 0.128 0.752 0.    0.009
  0.012]
 [0.    0.002 0.026 0.    0.    0.    0.    0.    0.014 0.    0.    0.957
  0.   ]
 [0.    0.    0.    0.    

In [ ]:
# lấy độ F1
f1 = []
for i in range(len(recall)):
  p = predict[i]
  r = recall[i]
  f1.append(2*p*r/(p+r))

print("Độ đo F1 của từng tag: ")
print(f1)
print(len(f1))

Độ đo F1 của từng tag: 
[0.999962956102982, 0.9466146848602989, 0.9712531832004718, 0.9147625657951163, 0.9966777408637874, 1.0, 1.0, 0.9933774834437086, 0.9259477220342308, 0.8040395985396029, nan, 0.9649754191825846, 0.989916972476514]
13
